In [58]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import time
import pandas as pd
import re
from tqdm import tqdm

In [113]:
def extract_number(text, return_type=float):
    """テキスト内から数字を取り出して返す（float or int）

    Parameters:
    ----------
    text : str
        数字が入ったテキスト
    return_type : type
        返す値の型。引数なしではfloat型となる

    Returns:
    ----------
    型：return_typeで選択した型
        textから取り出した数字を返す
    """
    matched_text = re.search(r'\d+(\.\d+)?', text)
    if matched_text:
        number = matched_text.group()
        if return_type == int:
            return int(float(number))  # floatへの変換後にintへ変換
        else:
            return float(number)
    else:
        return 0 if return_type == int else 0.0

d_list = []
url = 'https://suumo.jp/chintai/tokyo/sc_koto/?page={}'

In [114]:
for i in tqdm(range(1,15)):
    target_url = url.format(i)
    r = requests.get(target_url)
    time.sleep(1)
    soup = BeautifulSoup(r.text,"html.parser")
    contents = soup.find_all('div', {'class': 'cassetteitem'})
    for content in contents:
        
        # タイトル
        title = content.find('div', {'class': 'cassetteitem_content-title'}).get_text(strip=True) or None
        
        # 住所
        address = content.find('li', {'class': 'cassetteitem_detail-col1'}).get_text(strip=True) or None
        
        # アクセス
        access = ','.join([station.get_text(strip=True) for station in content.findAll('div', {'class': 'cassetteitem_detail-text'})])
        
        # 築年数
        age = content.find("li", {"class": "cassetteitem_detail-col3"}).find_all("div")[0].get_text(strip=True) or None
        
        # 総階数
        story = content.find("li", {"class": "cassetteitem_detail-col3"}).find_all("div")[1].get_text(strip=True) if len(content.find("li", {"class": "cassetteitem_detail-col3"}).find_all("div")) > 1 else None 
        
        # 階数、各金額、間取り、面積
        trtags = content.find('table', {'class': 'cassetteitem_other'}).find_all('tr', {'class': 'js-cassette_link'})
        for trtag in trtags:
            floor_tag, price, first_fee, capacity = trtag.find_all('td')[2:6]
            floor = re.sub(r'\s+', '', floor_tag.get_text())
            fee, management_fee = [li.get_text() for li in price.find_all('li')] or '0円'
            deposit, gratuity = [li.get_text() for li in first_fee.find_all('li')] or '0円'
            madori, menseki = [li.get_text() for li in capacity.find_all('li')]
            
            d = {
                'title': title,
                'address': address,
                'access': access,
                'age': age,
                'story': story,
                'floor': floor,
                'fee': fee,
                'management_fee': management_fee,
                'deposit': deposit,
                'gratuity': gratuity,
                'madori': madori,
                'menseki': menseki,
            }
            d_list.append(d)
df_suumo = pd.DataFrame(d_list)

100%|██████████| 14/14 [00:38<00:00,  2.73s/it]


In [115]:
df_suumo['fee'] = df_suumo['fee'].apply(extract_number)
df_suumo['management_fee'] = df_suumo['management_fee'].apply(extract_number)/10000
df_suumo['deposit'] = df_suumo['deposit'].apply(extract_number)
df_suumo['gratuity'] = df_suumo['gratuity'].apply(extract_number)
df_suumo['age'] = df_suumo['age'].apply(lambda x: "築1年" if x in ["新築", "築0年"] else x).apply(extract_number) # 「新築」「築0年」は1とする
df_suumo['story'] = df_suumo['story'].apply(extract_number)
df_suumo['floor'] = df_suumo['floor'].apply(extract_number)
df_suumo['menseki'] = df_suumo['menseki'].apply(extract_number)

In [116]:
# accessを取得し、「路線」「駅名」「徒歩分数」に分割し、それぞれ「access1_1」「access1_2」「access1_3」に格納する。アクセスは最大3件まで取得する
# df_suumoにカラム追加
for i in range(1, 4):
    for j in range(1, 4):
        df_suumo[f'access{i}_{j}'] = ''
# 行ごとにテキストを分解してカラムに格納
for index, row in df_suumo.iterrows():
    accesses = row['access'].split(',')[:3] # アクセス情報をコンマで分割し、最大3つまで取得
    for i, access in enumerate(accesses, start=1):
        match = re.match(r'([^/]+)/\s*([^ ]+)\s+歩(\d+)分', access.strip()) # 正規表現でテキストを解析
        if match:
            df_suumo.at[index, f'access{i}_1'] = match.group(1)
            df_suumo.at[index, f'access{i}_2'] = match.group(2)
            df_suumo.at[index, f'access{i}_3'] = match.group(3)

,title,address,access,age,story,floor,fee,management_fee,deposit,gratuity,...,menseki,access1_1,access1_2,access1_3,access2_1,access2_2,access2_3,access3_1,access3_2,access3_3
0,東京メトロ東西線 門前仲町駅 5階建 築1年,東京都江東区冬木,"東京メトロ東西線/門前仲町駅 歩8分,東京メトロ東西線/木場駅 歩10分,東京メトロ半蔵門線...",1.0,5.0,2.0,11.50,1.5,0.00,0.0,...,30.00,東京メトロ東西線,門前仲町駅,8,東京メトロ東西線,木場駅,10,東京メトロ半蔵門線,清澄白河駅,13
1,SUN VALLEY 日本橋永代通り,東京都江東区冬木,"東京メトロ東西線/門前仲町駅 歩8分,東京メトロ東西線/木場駅 歩10分,東京メトロ半蔵門線...",1.0,5.0,2.0,11.50,1.5,0.00,0.0,...,30.00,東京メトロ東西線,門前仲町駅,8,東京メトロ東西線,木場駅,10,東京メトロ半蔵門線,清澄白河駅,13
2,グランド・ガーラ木場,東京都江東区東陽１,"東京メトロ東西線/木場駅 歩6分,東京メトロ東西線/東陽町駅 歩6分,",4.0,13.0,2.0,9.60,0.9,9.60,19.2,...,25.51,東京メトロ東西線,木場駅,6,東京メトロ東西線,東陽町駅,6,,,
3,グランド・ガーラ木場,東京都江東区東陽１,"東京メトロ東西線/木場駅 歩6分,東京メトロ東西線/東陽町駅 歩6分,",4.0,13.0,2.0,9.80,0.9,9.80,19.6,...,25.51,東京メトロ東西線,木場駅,6,東京メトロ東西線,東陽町駅,6,,,
4,グランド・ガーラ木場,東京都江東区東陽１,"東京メトロ東西線/木場駅 歩6分,東京メトロ東西線/東陽町駅 歩6分,",4.0,13.0,8.0,9.85,0.9,9.85,19.7,...,25.51,東京メトロ東西線,木場駅,6,東京メトロ東西線,東陽町駅,6,,,


In [117]:
# Airdoorデータ取得
d_list = []
url = 'https://airdoor.jp/list?si=d-131083&p={}'

for i in tqdm(range(1,15)):
    target_url = url.format(i)
    r = requests.get(target_url)
    time.sleep(1) # 1秒ずつ
    soup = BeautifulSoup(r.text,"html.parser")
    contents = soup.find_all('div', {'class': 'PropertyPanel_propertyPanel__8oJ13'}) or None
    for content in contents:
        
        # タイトル
        title = content.find('div', {'class': 'PropertyPanelBuilding_buildingTitle__tuPqN'}).get_text(strip=True) or None
        
        # 住所
        building_info = content.find_all('div', {'class': 'PropertyPanelBuilding_buildingInformationSection__deSLp'})
        address = building_info[0].find('p', {'class': 'is-mt5'}).get_text(strip=True) or None
        access = ', '.join(p.get_text() for p in building_info[0].find_all('p', {'class': False})) or None        
        
        # 築年数、総階数
        p_tags = building_info[1].find_all('p')
        age = re.search(r'\((.*?)\)', p_tags[0].get_text()).group(1) or '築0年'
        story = p_tags[1].get_text(strip=True)

        
        # 階数、間取り、面積
        roomItems = content.findAll('a', {'class': 'PropertyPanelRoom_roomItem__95jRr'})
        for roomItem in roomItems:
            p_tag_text = roomItem.find('span', {'class': 'is-ml5'}).get_text(strip=True)
            room_number, madori, menseki, hogaku = [part.strip() for part in p_tag_text.split('/')]
            
            # 階数
            floor = re.findall(r'\d+', room_number)[0][:-2] if re.findall(r'\d+', room_number) and len(re.findall(r'\d+', room_number)[0]) > 2 else '1'
            
            # 家賃、管理費
            div_text = roomItem.find('div', {'class': 'PropertyPanelRoom_rentPrice__XdPUp'}).text
            fee = div_text.split()[0].replace(',', '') or '0円'
            management_fee = div_text.split()[1].replace(',', '') or '0円'
            
            # 敷金、礼金
            div = roomItem.find('div', {'class': 'PropertyPanelRoom_initialPrices__d90C3'})
            deposit = div.find_all('li')[0].get_text(strip=True) or '0円'
            gratuity = div.find_all('li')[1].get_text(strip=True) or '0円'
            
            d = {
                'title': title,
                'address': address,
                'access': access,
                'age': age,
                'story': story,
                'floor': floor,
                'fee': fee,
                'management_fee': management_fee,
                'deposit': deposit,
                'gratuity': gratuity,
                'madori': madori,
                'menseki': menseki,
            }
            
            d_list.append(d)
df_airdoor = pd.DataFrame(d_list)

100%|██████████| 14/14 [00:28<00:00,  2.07s/it]


In [118]:
df_airdoor['title'] = df_airdoor['title'].str.replace(r'【.*?】', '', regex=True)
df_airdoor['fee'] = df_airdoor['fee'].apply(extract_number)/10000
df_airdoor['management_fee'] = df_airdoor['management_fee'].apply(extract_number)/10000
df_airdoor['deposit'] = df_airdoor['deposit'].apply(lambda x: "0円" if x in ["無料"] else x).apply(extract_number)
df_airdoor['gratuity'] = df_airdoor['gratuity'].apply(lambda x: "0円" if x in ["無料"] else x).apply(extract_number)
df_airdoor['age'] = df_airdoor['age'].apply(lambda x: "築1年" if x in ["新築", "築0年"] else x).apply(extract_number)
df_airdoor['story'] = df_airdoor['story'].apply(extract_number)
df_airdoor['floor'] = df_airdoor['floor'].apply(extract_number)
df_airdoor['menseki'] = df_airdoor['menseki'].apply(extract_number)

In [119]:
# accessを取得し、「路線」「駅名」「徒歩分数」に分割し、それぞれ「access1_1」「access1_2」「access1_3」に格納する。アクセスは最大3件まで取得する
# df_airdoorにカラム追加
for i in range(1, 4):
    for j in range(1, 4):
        df_airdoor[f'access{i}_{j}'] = ''
df_airdoor.head()
# 行ごとにテキストを分解してカラムに格納
for index, row in df_airdoor.iterrows():
    accesses = row['access'].split(',')[:3] # アクセス情報をコンマで分割し、最大3つまで取得
    for i, access in enumerate(accesses, start=1):
        match = re.match(r'(.+?)\s+(.+?)\s+徒歩(\d+)分', access.strip()) # 正規表現でテキストを解析
        if match:
            df_airdoor.at[index, f'access{i}_1'] = match.group(1)
            df_airdoor.at[index, f'access{i}_2'] = match.group(2)
            df_airdoor.at[index, f'access{i}_3'] = match.group(3)

,title,address,access,age,story,floor,fee,management_fee,deposit,gratuity,...,menseki,access1_1,access1_2,access1_3,access2_1,access2_2,access2_3,access3_1,access3_2,access3_3
0,グランカーサ大島,東京都江東区大島４丁目,"都営新宿線 大島駅 徒歩3分, 都営新宿線 西大島駅 徒歩7分",1.0,12.0,7.0,10.0,1.0,0.0,0.0,...,25.41,都営新宿線,大島駅,3,都営新宿線,西大島駅,7,,,
1,グランカーサ大島,東京都江東区大島４丁目,"都営新宿線 大島駅 徒歩3分, 都営新宿線 西大島駅 徒歩7分",1.0,12.0,12.0,23.3,1.5,0.0,0.0,...,50.82,都営新宿線,大島駅,3,都営新宿線,西大島駅,7,,,
2,プレール・ドゥーク木場公園ＷＥＳＴ,東京都江東区平野３丁目,"東京メトロ半蔵門線 清澄白河駅 徒歩8分, 都営大江戸線 清澄白河駅 徒歩11分",1.0,8.0,3.0,12.2,1.0,0.0,0.0,...,25.25,東京メトロ半蔵門線,清澄白河駅,8,都営大江戸線,清澄白河駅,11,,,
3,アーバネックス森下ＷＥＳＴ,東京都江東区新大橋２丁目,"都営新宿線 森下駅 徒歩6分, ＪＲ総武線 両国駅 徒歩11分",1.0,12.0,12.0,13.2,1.0,13.2,0.0,...,25.97,都営新宿線,森下駅,6,ＪＲ総武線,両国駅,11,,,
4,アーバネックス森下ＷＥＳＴ,東京都江東区新大橋２丁目,"都営新宿線 森下駅 徒歩6分, ＪＲ総武線 両国駅 徒歩11分",1.0,12.0,5.0,11.7,1.0,11.7,0.0,...,25.97,都営新宿線,森下駅,6,ＪＲ総武線,両国駅,11,,,


In [120]:
df = pd.concat([df_suumo, df_airdoor])
df_dropped = df.drop_duplicates(['address', 'age', 'story', 'floor', 'madori', 'menseki'])
df_dropped.shape

(1245, 21)

In [121]:
import sqlite3

# SQLiteデータベースへの接続
conn = sqlite3.connect('/Users/ryosukeinoue/Library/CloudStorage/GoogleDrive-ryosuke.inoue0314@gmail.com/マイドライブ/00_本データ/31_Tech0/Step3/techone/techonedata.db')

# DataFrameをSQLiteデータベースにインポート
df_dropped.to_sql('techonedata', conn, if_exists='replace', index=False)

# 接続を閉じる
conn.close()